# Taller 4

Autor: Guillermo Luigui Ubaldo Nieto Angarita

Rol: Estudiante en Maestría en Analítica Estrategica de Datos

Instrucciones: 
- Vectorizar con TF-IDF, Glove, Word2Vec
- Implementar técnicas de Ngrams
- Calcular la distancia del Coseno con los 3 vectorizaciones 
- representar mediante Mapa de calor los resultados de los distancia del Coseno
- Calcular y representar el Top Modeling Mediante LDA.

## 1. Limpieza del texto

### 1.1 Cargar librerías

In [78]:
import pdfplumber  # Para extraer texto de PDFs
import re  # Para trabajar con expresiones regulares
import spacy  # Para procesamiento de lenguaje natural (NLP)
import unicodedata  # Para normalizar caracteres Unicode
import stanza
import nltk
import fitz  # PyMuPDF
import docx2txt
import requests
import os
import gzip
import numpy as np

from transformers import AutoTokenizer
from nltk.corpus import stopwords
from docx import Document
from nltk.stem import SnowballStemmer
from flair.data import Sentence
from flair.models import SequenceTagger
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm.notebook import tqdm
from gensim.models import Word2Vec
from flair.models import Lemmatizer
from huggingface_hub import login

In [79]:
# Introduce tu token aquí
token = os.getenv("FLAIR_TOKEN")

login(token=token)

### 1.2 Descargar los tokenizadores

In [63]:
nltk.download('punkt')
stanza.download('es')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


2025-09-11 00:38:02 INFO: Downloaded file to C:\Users\guill\stanza_resources\resources.json
2025-09-11 00:38:02 INFO: Downloading default packages for language: es (Spanish) ...
2025-09-11 00:38:03 INFO: File exists: C:\Users\guill\stanza_resources\es\default.zip
2025-09-11 00:38:06 INFO: Finished downloading models and saved to C:\Users\guill\stanza_resources


In [64]:
nlp_stanza = stanza.Pipeline("es")
nlp_spacy = spacy.load("es_core_news_sm")  #cargar el modelo en español
tokenizer_bert = AutoTokenizer.from_pretrained("dccuchile/bert-base-spanish-wwm-cased")

2025-09-11 00:38:06 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-09-11 00:38:06 INFO: Downloaded file to C:\Users\guill\stanza_resources\resources.json
2025-09-11 00:38:07 INFO: Loading these models for language: es (Spanish):
| Processor    | Package           |
------------------------------------
| tokenize     | combined          |
| mwt          | combined          |
| pos          | combined_charlm   |
| lemma        | combined_nocharlm |
| constituency | combined_charlm   |
| depparse     | combined_charlm   |
| sentiment    | tass2020_charlm   |
| ner          | conll02           |

2025-09-11 00:38:07 INFO: Using device: cpu
2025-09-11 00:38:07 INFO: Loading: tokenize
2025-09-11 00:38:07 INFO: Loading: mwt
2025-09-11 00:38:07 INFO: Loading: pos
2025-09-11 00:38:09 INFO: Loading: lemma
2025-09-11 00:38:10 INFO: Loading: constituency
2025-09-11 00:38:10 INFO: Loading: depparse
2025-09-11 00:38:11 INFO: Loading: sentiment
2025-09-11 00:38:11 INFO: Loading: ner
2025-09-11 00:38:13 INFO: Done loading processors!


### 1.3 Descargar Stopwords en Español

In [65]:
nltk.download('stopwords')
spanish_stopwords = set(stopwords.words('spanish'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\guill\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1.4 Cargar texto

### 1.5 Leer los Documentos Docx

In [66]:
doc_gabriel_garcia = Document("texts\Discuros Gabriel García Marquez.docx")
for para in doc_gabriel_garcia.paragraphs:
    print(para.text)

Antonio Pigafetta, un navegante florentino que acompañó a Magallanes en el primer viaje alrededor del mundo, escribió a su paso por nuestra América meridional una crónica rigurosa que sin embargo parece una aventura de la imaginación. Contó que había visto cerdos con el ombligo en el lomo, y unos pájaros sin patas cuyas hembras empollaban en las espaldas del macho, y otros como alcatraces sin lengua cuyos picos parecían una cuchara. Contó que había visto un engendro animal con cabeza y orejas de mula, cuerpo de camello, patas de ciervo y relincho de caballo. Contó que al primer nativo que encontraron en la Patagonia le pusieron enfrente un espejo, y que aquel gigante enardecido perdió el uso de la razón por el pavor de su propia imagen.
Este libro breve y fascinante, en el cual ya se vislumbran los gérmenes de nuestras novelas de hoy, no es ni mucho menos el testimonios más asombroso de nuestra realidad de aquellos tiempos. Los cronistas de Indias nos legaron otros incontables. Eldorad

In [67]:
doc_albert_camus = Document("texts\Discurso Albert Camus.docx")
for para in doc_albert_camus.paragraphs:
    print(para.text)

«Al recibir la distinción con que ha querido honrarme su libre Academia, mi gratitud es más profunda cuando evalúo hasta qué punto esa recompensa sobrepasa  mis méritos personales. Todo hombre, y con mayor razón todo artista, desea que se reconozca lo que es o quiere ser. Yo también lo deseo. Pero al conocer su decisión me fue imposible no comparar su resonancia con lo que realmente soy. ¿Cómo un hombre, casi joven todavía, rico sólo por sus dudas, con una obra apenas desarrollada, habituado a vivir en la soledad del trabajo o en el retiro de la amistad, podría recibir, sin una especie de pánico, un galardón que le coloca de pronto, y solo, a plena luz? ¿Con qué ánimo podía recibir ese honor al tiempo que, en tantos sitios, otros escritores, algunos de los más grandes, están reducidos al silencio y cuando, al mismo tiempo, su tierra natal conoce una desdicha incesante?
He sentido esa inquietud, y ese malestar. Para recobrar mi paz interior me ha sido necesario ponerme de acuerdo con un

In [68]:
doc_bob_dylan = Document("texts\Discurso Bob Dylan.docx")
for para in doc_bob_dylan.paragraphs:
    print(para.text)

Discurso en el banquete del Nobel, 10 de diciembre de 2016
Buenas noches a todos. Les envío un cordial saludo a los miembros de la Academia Sueca y a todos los distinguidos invitados presentes esta noche.
Lamento no poder estar con ustedes en persona, pero sepan que estoy definitivamente con ustedes en espíritu y me siento honrado de recibir un premio tan prestigioso. Recibir el Premio Nobel de Literatura es algo que nunca podría haber imaginado ni visto venir. Desde muy temprana edad, he estado familiarizado con, leído y absorbido las obras de quienes fueron considerados merecedores de tal distinción: Kipling , Shaw , Thomas Mann , Pearl Buck , Albert Camus , Hemingway . Estos gigantes de la literatura cuyas obras se enseñan en el aula, se albergan en bibliotecas de todo el mundo y se habla de ellas con tonos reverentes siempre han causado una profunda impresión. Que ahora me una a los nombres de tal lista es realmente indescriptible.
No sé si estos hombres y mujeres alguna vez pensar

In [69]:
doc_vargas_llosa = Document("texts\Discurso Vargas Llosa.docx")
for para in doc_vargas_llosa.paragraphs:
    print(para.text)

Elogio de la lectura y la ficción
Aprendí a leer a los cinco años, en la clase del hermano Justiniano, en el Colegio de la Salle, en Cochabamba (Bolivia). Es la cosa más importante que me ha pasado en la vida. Casi setenta años después recuerdo con nitidez cómo esa magia, traducir las palabras de los libros en imágenes, enriqueció mi vida, rompiendo las barreras del tiempo y del espacio y permitiéndome viajar con el capitán Nemo veinte mil leguas de viaje submarino, luchar junto a d’Artagnan, Athos, Portos y Aramís contra las intrigas que amenazan a la Reina en los tiempos del sinuoso Richelieu, o arrastrarme por las entrañas de París, convertido en Jean Valjean, con el cuerpo inerte de Marius a cuestas.
La lectura convertía el sueño en vida y la vida en sueño y ponía al alcance del pedacito de hombre que era yo el universo de la literatura. Mi madre me contó que las primeras cosas que escribí fueron continuaciones de las historias que leía pues me apenaba que se terminaran o quería en

### 1.6 Eliminación de elementos no deseados

In [70]:
def extract_text_without_unwanted_elements(text):
    text = re.sub(r"[^a-zA-Z0-9áéíóúÁÉÍÓÚüÜñÑ\s]", "", text)
    return text

In [86]:
def extract_text_without_unwanted_elements_sentences(text):
    """
    Clean Spanish text but keep sentence delimiters (.!?¡¿) and newlines.
    """
    # Keep Spanish letters, digits, spaces, newlines, and sentence delimiters
    text = re.sub(r"[^a-zA-Z0-9áéíóúÁÉÍÓÚüÜñÑ \n\.\!\?\¡\¿]", "", text)
    
    # Optional: normalize multiple newlines into a single one
    text = re.sub(r"\n+", "\n", text)
    
    return text


### 1.6 Conversión a minúsculas

In [72]:
def convertToLowerCase(text):
    return text.lower()

### 1.7 Transformación UNICODE

In [73]:
def unicodeTransformation(text, method):
    return unicodedata.normalize(method, text)

### 1.8 Tokenización

In [74]:
def stanza_tokenization_words(text):
    doc = nlp_stanza(text)
    return [word.text for sent in doc.sentences for word in sent.words]

In [75]:
def stanza_tokenization_sentences(text):
    """
    Tokenize text into words using Stanza (Spanish).
    Returns: list of tokens per sentence (nested list).
    """
    doc = nlp_stanza(text)
    return [[word.text for word in sent.words] for sent in doc.sentences]

### 1.9 Eliminación de stop words

In [76]:
# --- Function for spaCy ---
def remove_stopwords_spacy(text):
    """
    Return Spanish text without stop words using spaCy
    """
    nlp = spacy.load("es_core_news_sm")
    doc = nlp(text)
    filtered = [token.text for token in doc if not token.is_stop]
    return " ".join(filtered)

### 1.10 Lematización

In [82]:
def lemmatize_with_stanza(text):
    """
    Lemmatize text using Stanza (Spanish pipeline).
    Returns a flat list of lemmas.
    """
    doc = nlp_stanza(text)
    return [word.lemma for sent in doc.sentences for word in sent.words]


In [83]:
def lemmatize_with_stanza_sentences(text):
    """
    Lemmatize text using Stanza.
    Splits into sentences and returns list of lists of lemmas.
    """
    doc = nlp_stanza(text)
    
    lemmatized_sentences = []
    for sent in doc.sentences:
        lemmas = [word.lemma for word in sent.words]
        lemmatized_sentences.append(lemmas)
    
    return lemmatized_sentences


### 1.11 Stemming

In [ ]:
# --- Function 2: Stemming with NLTK ---
def stem_spanish(text):
    """
    Stem Spanish text (no stopword removal)
    """
    stemmer = SnowballStemmer("spanish")
    tokens = nltk.word_tokenize(text, language="spanish")
    stems = [stemmer.stem(word) for word in tokens if word.isalpha()]
    return stems

### 1.12 Identificar NERs

In [ ]:
def apply_ner(text):
    sentence = Sentence(text)
    tagger.predict(sentence)
    entities = [(entity.text, entity.labels[0].value) for entity in sentence.get_spans('ner')]
    return entities

### 1.13 Procedimiento de Textos

In [ ]:
def text_processing_pipeline_words(text):
    text_unicode = unicodeTransformation(text, "NFKD")

    text_cleaned = extract_text_without_unwanted_elements(text_unicode)

    text_lower_case = convertToLowerCase(text_cleaned)

    text_without_stop_words = remove_stopwords_spacy(text_lower_case)

    tokens = stanza_tokenization_words(text_without_stop_words)
    
    lammes = lemmatize_with_stanza(' '.join(tokens))
    
    return ' '.join(lammes)

In [84]:
def text_processing_pipeline_sentences(text):
    # Step 1: Unicode normalization
    text_unicode = unicodeTransformation(text, "NFKD")

    # Step 2: Clean unwanted elements
    text_cleaned = extract_text_without_unwanted_elements_sentences(text_unicode)

    # Step 3: Lowercase
    text_lower_case = convertToLowerCase(text_cleaned)

    # Step 4: Remove stopwords
    text_without_stop_words = remove_stopwords_spacy(text_lower_case)

    # Step 5: Lemmatization with Stanza (handles sentence + word tokenization itself)
    lemmatized_sentences = lemmatize_with_stanza_sentences(text_without_stop_words)

    return lemmatized_sentences

### 1.14 Ejecutar Limpieza

In [ ]:
gabriel_garcia_marquez_text = docx2txt.process("texts/Discuros Gabriel García Marquez.docx")
albert_camus_text = docx2txt.process("texts/Discurso Albert Camus.docx")
bob_dylan_text = docx2txt.process("texts/Discurso Bob Dylan.docx")
vargas_llosa_text = docx2txt.process("texts/Discurso Vargas Llosa.docx")

In [ ]:
gabriel_garcia_marquez_lammes = text_processing_pipeline_words(gabriel_garcia_marquez_text)

2025-09-10 20:58:19,908 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [ ]:
albert_camus_lammes = text_processing_pipeline_words(albert_camus_text)

2025-09-10 20:58:40,264 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [ ]:
bob_dylan_lammes = text_processing_pipeline_words(bob_dylan_text)

2025-09-10 20:58:51,193 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [ ]:
vargas_llosa_lammes = text_processing_pipeline_words(vargas_llosa_text)

2025-09-10 20:59:33,401 SequenceTagger predicts: Dictionary with 53 tags: <unk>, O, UH, ,, VBD, PRP, VB, PRP$, NN, RB, ., DT, JJ, VBP, VBG, IN, CD, NNS, NNP, WRB, VBZ, WDT, CC, TO, MD, VBN, WP, :, RP, EX, JJR, FW, XX, HYPH, POS, RBR, JJS, PDT, NNPS, RBS, AFX, WP$, -LRB-, -RRB-, ``, '', LS, $, SYM, ADD


In [ ]:
gabriel_garcia_marquez_sentences = text_processing_pipeline_sentences(albert_camus_text)

In [90]:
albert_camus_sentences = text_processing_pipeline_sentences(gabriel_garcia_marquez_text)

In [91]:
bob_dylan_sentences = text_processing_pipeline_sentences(bob_dylan_text)

In [92]:
vargas_llosa_sentences = text_processing_pipeline_sentences(vargas_llosa_text)

## 2. Vectorización

### 2.1 Vectorización con TF-IDF

In [93]:
docs = [
    gabriel_garcia_marquez_lammes,
    albert_camus_lammes,
    bob_dylan_lammes,
    vargas_llosa_lammes
]

In [94]:
# TF-IDF Vectorizer (Spanish stopwords optional)
vectorizer = TfidfVectorizer(stop_words=None)  # or provide custom list
X = vectorizer.fit_transform(docs)

### 2.2 Vectorización con GloVe

In [95]:
url = "http://dcc.uchile.cl/~jperez/word-embeddings/glove-sbwc.i25.vec.gz"
local_path = "glove-sbwc.i25.vec.gz"

if not os.path.exists(local_path) or os.path.getsize(local_path) == 0:
    print(f"Downloading {local_path} ...")
    response = requests.get(url, stream=True)
    response.raise_for_status()
    
    total_size = int(response.headers.get("content-length", 0))
    block_size = 1024  # 1 KB
    
    with open(local_path, "wb") as file, tqdm(
        total=total_size, unit="iB", unit_scale=True
    ) as bar:
        for chunk in response.iter_content(block_size):
            file.write(chunk)
            bar.update(len(chunk))
    
    print("✅ Download complete.")
else:
    print(f"✅ File already exists ({os.path.getsize(local_path)} bytes). No download needed.")

✅ File already exists (949886421 bytes). No download needed.


In [96]:
# Step 2: Load embeddings into a dictionary (only the words we care about)
def load_embeddings(file_path, vocab=None):
    embeddings = {}
    with gzip.open(file_path, "rt", encoding="utf-8") as f:
        for line in f:
            parts = line.strip().split()
            word = parts[0]
            if vocab is None or word in vocab:
                try:
                    vector = list(map(float, parts[1:]))
                    embeddings[word] = vector
                except ValueError:
                    continue
    return embeddings

In [ ]:
gabriel_garcia_marquez_lamme_tokens = gabriel_garcia_marquez_lammes.split()

In [ ]:
albert_camus_lamme_tokens = albert_camus_lammes.split()

In [ ]:
bob_dylan_lamme_tokens = bob_dylan_lammes.split()

In [ ]:
vargas_llosa_lamme_tokens = vargas_llosa_lammes.split()

In [ ]:
# Step 3: Load embeddings only for these tokens
gabriel_garcia_marquez_vocab = set(gabriel_garcia_marquez_lamme_tokens)
gabriel_garcia_marquez_vocab_embeddings = load_embeddings(local_path, gabriel_garcia_marquez_vocab)

In [ ]:
albert_camus_vocab = set(albert_camus_lamme_tokens)
albert_camus_embeddings = load_embeddings(local_path, albert_camus_vocab)

In [ ]:
bob_dylan_vocab = set(bob_dylan_lamme_tokens)
bob_dylan_embeddings = load_embeddings(local_path, bob_dylan_vocab)

In [ ]:
vargas_llosa_vocab = set(vargas_llosa_lamme_tokens)
vargas_llosa_embeddings = load_embeddings(local_path, vargas_llosa_vocab)

In [ ]:
# Step 4: Vectorize text (average embedding)
def vectorize_text(tokens, embeddings):
    vecs = [embeddings[word] for word in tokens if word in embeddings]
    if not vecs:
        return np.zeros(25)  # fallback if no embeddings found
    return np.mean(vecs, axis=0)

In [ ]:
gabriel_garcia_marquez_vector = vectorize_text(gabriel_garcia_marquez_lamme_tokens, gabriel_garcia_marquez_vocab_embeddings)

print("Vector shape:", gabriel_garcia_marquez_vector.shape)
print("Vector (first 5 dims):", gabriel_garcia_marquez_vector[:5])

Vector shape: (300,)
Vector (first 5 dims): [-0.01470105  0.03982766 -0.00598356 -0.00013167  0.03406841]


In [ ]:
albert_camus_vector = vectorize_text(albert_camus_lamme_tokens, albert_camus_embeddings)

print("Vector shape:", albert_camus_vector.shape)
print("Vector (first 5 dims):", albert_camus_vector[:5])

Vector shape: (300,)
Vector (first 5 dims): [-0.07566411  0.08753591 -0.00210951  0.03758463 -0.00879787]


In [ ]:
bob_dylan_vector = vectorize_text(bob_dylan_lamme_tokens, bob_dylan_embeddings)

print("Vector shape:", bob_dylan_vector.shape)
print("Vector (first 5 dims):", bob_dylan_vector[:5])

Vector shape: (300,)
Vector (first 5 dims): [-0.02437542  0.12023954  0.03665135  0.07913365  0.12269059]


In [ ]:
vargas_llosa_vector = vectorize_text(vargas_llosa_lamme_tokens, vargas_llosa_embeddings)

print("Vector shape:", vargas_llosa_vector.shape)
print("Vector (first 5 dims):", vargas_llosa_vector[:5])

Vector shape: (300,)
Vector (first 5 dims): [-0.05418063  0.08038231 -0.02178696  0.02122399  0.04492896]


### 2.3 Vectorización con Word2Vec

In [ ]:
gabriel_garcia_marquez_sentences
albert_camus_sentences
bob_dylan_sentences
vargas_llosa_sentences

In [97]:
# Train Word2Vec model
model = Word2Vec(
    gabriel_garcia_marquez_sentences,
    vector_size=25,   # embedding size (can be 25, 50, 100…)
    window=5,         # context window
    min_count=1,      # keep all words
    workers=4,
    sg=1              # 1 = skip-gram, 0 = CBOW
)

In [101]:
# Example: get vector for a word
print("Vector for 'soledad':")
print(model.wv['soledad'])

# Example: find most similar words
print("\nMost similar to 'soledad':")
print(model.wv.most_similar('soledad'))

Vector for 'soledad':
[ 0.01351639 -0.03284274  0.03206305  0.03240193 -0.03818129  0.00933546
  0.04025585 -0.02815499 -0.03081796 -0.03304289  0.03578829 -0.003574
  0.03733782 -0.03046647  0.01359323  0.0143363   0.00197631  0.00990706
 -0.03351754 -0.03601529  0.00985445  0.0251933   0.03260829  0.02194987
 -0.0016914 ]

Most similar to 'soledad':
[('cambio', 0.6847589015960693), ('hijo', 0.5102234482765198), ('escribir', 0.505395233631134), ('él', 0.4928467869758606), ('distancia', 0.48601725697517395), ('persona', 0.47745034098625183), ('discurso', 0.47743070125579834), ('virtud', 0.4653416872024536), ('imponer', 0.46123090386390686), ('.', 0.458726167678833)]


## Implementar técnicas de N-grams